<a href="https://colab.research.google.com/github/lingxiaoW/Ph.D._Advising_LX/blob/main/Hoang/createcustomdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load and Preprocessing CSV file**

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

#load the CSV file
file_path = 'path/to/your/file.csv'
data = pd.read_csv(file_path)

#assume the time series data is in a column named 'value'
time_series = data['value'].values.astype(np.float32)

#handling missing value (if any)
time_series = time_series[~np.isnan(time_series)]

#normalize the data (important for neural networks)
mean = np.mean(time_series)
std = np.std(time_series)
time_series = (time_series - mean) / std   #standard scaling

**Create sequence for Time series**

In [ ]:
def create_sequence(data, window_size):
    x, y = [], []
    for i in range(len(data)):
        #find the end of this pattern
        end_ix = i + window_size
        #check if we are beyond the sequence
        if end_ix > len(data)-1:
            break
        #gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        x.append(seq_x)
        y.append(seq_y)
    return array(x), array(y)

window_size = 4
#split into samples
x, y = create_sequence(data, window_size)
#summarize the data
for i in range(len(x)):
  print(x[i], y[i])

**Create custom PyTorch Dataset**

In [ ]:
import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]
#create the dataset
dataset = TimeSeriesDataset(x, y)

**Split the Data into Training and Testing sets**

In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

#create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)